In [1]:
import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', 500)

import dask
import dask.dataframe as dd

import keggler as kg

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import gc
gc.enable()

import warnings
warnings.simplefilter(action='ignore', category=Warning)

import os, psutil

# Set up a logger to dump messages to both log file and notebook
import logging as logging
def ini_log(filename):
    logger = logging.getLogger(__name__)
    ## avoid multiple printouts due to same handlers added several times
    if not logger.handlers:
        logger.setLevel(logging.DEBUG)

        handlers = [logging.StreamHandler(None), logging.FileHandler(filename, 'a')]

        fmt=logging.Formatter('%(asctime)-15s: %(levelname)s  %(message)s')
        for h in handlers:
            h.setFormatter(fmt)
            logger.addHandler(h)
    return logger
        
log = ini_log('out.log')

In [8]:
from multiprocessing.pool import ThreadPool
import dask
dask.config.set(pool=ThreadPool(5))

In [3]:
data_path = 'data/' # point this to your data folder
test_path = data_path + 'test_set/'
submission_path = data_path + 'submissions/'
test_input_logs = sorted(glob.glob(test_path + "log_input_*.csv.gz"))
test_prehi_logs = sorted(glob.glob(test_path + "log_prehistory_*.csv.gz"))

In [4]:
dd_tst = dd.read_csv(test_prehi_logs[:],
                     usecols=[0,1,2,5], 
                     compression='gzip')

In [13]:
# dd_tst.head(50)

In [6]:
last_played_track = dd_tst.groupby('session_id').last()

In [10]:
sub = last_played_track['skip_2'].astype(np.uint8).astype(str) * (last_played_track['session_length'] - last_played_track['session_position'])

In [11]:
sub.to_csv(submission_path+'/sub_last_played_*.csv', index=False)

['data/submissions//sub_last_played_0.csv']

Single GB operation in 3 threads: `7m`, memory jumps around

In [5]:
def get_last_test_track_per_session(df_, col_):
    return df_.groupby(col_).last()

In [6]:
last_played_track = dd_tst.map_partitions(get_last_test_track_per_session, col_='session_id')

In [7]:
sub = last_played_track['skip_2'].astype(np.uint8).astype(str) * (last_played_track['session_length'] - last_played_track['session_position'])

In [9]:
sub.repartition(npartitions=1).to_csv(submission_path+'/sub_last_played_mp_*.csv', index=False)

['data/submissions//sub_last_played_mp_0.csv']

GB operation on each partition separately:
- 3 threads `8 m`;
- 5 threads `6 m 45`